In [1]:
import cv2
import numpy as np
import imutils

class Panorama:

    # the blend types
    NO_BLEND = 1
    BLEND_WEIGHED  = 2
    BOTH = 3
    
    # we make the weighed average of the two images
    # the idea roughly comes from here: http://bigwww.epfl.ch/publications/thevenaz0701.pdf
    # the above link found here: https://stackoverflow.com/questions/36386968/image-stitching-methods-to-remove-seams-for-stitched-image
    @classmethod
    def blendWeighed(cls, imgA, imgB, thresh=3):
        result = np.zeros([imgA.shape[0], imgA.shape[1] ,3], dtype=np.uint8)
        distA = cv2.distanceTransform(cv2.cvtColor(imgA, cv2.COLOR_BGR2GRAY), cv2.DIST_L2, 5)
        distB = cv2.distanceTransform(cv2.cvtColor(imgB, cv2.COLOR_BGR2GRAY), cv2.DIST_L2, 5)
        #cv2.imwrite('distA.png', distA)
        #cv2.imwrite('distB.png', distB)
        # for every pixel within the result image (that has the same dimensions as the imgA/imgB!)
        for i in range(imgA.shape[0]):
            for j in range(imgA.shape[1]):
                # if the current pixel is "out of" the first image (or is just black)
                if np.all(imgA[i,j] < thresh):
                    result[i,j] = imgB[i,j]
                # if the current pixel is "out of" the second image (or is just black)
                elif np.all(imgB[i,j] < thresh):
                    result[i,j] = imgA[i,j]
                # else, we make the weighted sum and do the magic
                # notice that here distA[i,j]+distB[i,j] can't be zero, since
                # the upper cases covered that
                else:
                    result[i,j] = (imgA[i,j]*distA[i,j] + imgB[i,j]*distB[i,j]) / (distA[i,j]+distB[i,j])
        
        
        return result
   
    # naive merge; seams are to be seen, probably
    @classmethod
    def merge(cls, imgDest, imgSrc):
        # and just paste the second image onto the first one
        result = np.copy(imgDest)
        result[0:imgSrc.shape[0], 0:imgSrc.shape[1]] = imgSrc
        return result
    
    # obtaining the keypoints and their features for an image
    @classmethod
    def detectAndDescribe(cls, image):
        # convert the image to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # detect and extract features from the image
        descriptor = cv2.xfeatures2d.SIFT_create()
        (kps, features) = descriptor.detectAndCompute(image, None)
        return (kps, features)
    
    # the method for drawing matches in the image left and right, given the 
    # keypoints and matches
    @classmethod
    def drawMatches(cls, right, kpsR, left, kpsL, matches):
        matchImg = None
        matchImg = cv2.drawMatches(right, kpsR, left, kpsL, matches, matchImg)
        return matchImg
    @classmethod
    def drawKeypoints(cls, image, keypoints):
        draw = None
        draw = cv2.drawKeypoints(image, keypoints, draw)
        return draw
    
    @classmethod
    def transform(cls, matches, kpsR, kpsL):
        matchesIdx = [(m.queryIdx, m.trainIdx) for m in matches[:10]]
        matchesCoordR = np.float32([kpsR[mind[0]].pt for mind in matchesIdx])
        matchesCoordL = np.float32([kpsL[mind[1]].pt for mind in matchesIdx])
        (H, mask) = cv2.findHomography(matchesCoordR, matchesCoordL, cv2.RANSAC)
        
        imgDest = None
        imgDest = cv2.warpPerspective(right, H, (right.shape[1]+left.shape[1], left.shape[0] + right.shape[0]))
        
        return imgDest
        
    @classmethod
    def makePanorama(cls, right, left, drawKeypoints=False, drawMatches=False, blend_type=NO_BLEND):
        # the result is a dictionary that contains all the possible images derived:
        # result with no blending
        # result with blending
        # right image with keypoints drawn
        # left image with keypoints drawn
        # both images with matches drawn
        result = {'resNoBlend':None, 'resBlend':None, 'kpsR':None, 'kpsL':None, 'matches':None}
        
        # first, we find the keypoints in the images and their features 
        (kpsR, ftsR) = cls.detectAndDescribe(right)
        (kpsL, ftsL) = cls.detectAndDescribe(left)
        
        # if we should draw the keypoints, we draw them and put in the dictionary
        if drawKeypoints:
            result['kpsR'] = cls.drawKeypoints(right, kpsR)
            result['kpsL'] = cls.drawKeypoints(left, kpsL)
        
        
        
        #orb = cv2.ORB_create()
        
        # we find the matcher for the images and sort them by distance (by relevance)
        bf = cv2.BFMatcher(cv2.NORM_L1,crossCheck=False)
        matches = bf.match(ftsR,ftsL)
        matches = sorted(matches, key=lambda x:x.distance)
        
        # if we should draw the matches, we draw them and put the result in the dictionary
        if drawMatches:
            result['matches'] = cls.drawMatches(right, kpsR, left, kpsL, matches[:10])
        
        # we transform the right image
        newDst = cls.transform(matches, kpsR, kpsL)
        #cv2.imwrite('newDst.png', Panorama.crop(newDst))
       
        # we pad the left image with black background so that the two images have matching dimensions
        newSrc = np.zeros([left.shape[0] + right.shape[0], right.shape[1]+left.shape[1] ,3],dtype=np.uint8)
        newSrc[0:left.shape[0], 0:left.shape[1]] = left
        #cv2.imwrite('newSrc.png', newSrc)
        
        # now, both images that we use are in newDst (right) and newSrc (left)
        
        # depending on the type of blending chosen, we blend (or don't) and put the results 
        # in the dictionary
        if blend_type == cls.NO_BLEND or blend_type == cls.BOTH:
            # since the newSrc is now padded, we use left!
            result['resNoBlend'] = cls.merge(newDst, left)
        if blend_type == cls.BLEND_WEIGHED or blend_type == cls.BOTH:
            result['resBlend'] = cls.blendWeighed(newDst, newSrc)
        
        return result
    # cropping doesn't remove ALL the black parts, just the maximal amount that
    # keeps all the pixels of the stitched images 
    @classmethod
    def crop(cls,img):
        imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(imgray, 10, 10, 10)
        im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        maxa = 0
        maxi = 0
        for i in range(len(contours)):
            tmpmax = cv2.contourArea(contours[i])
            if maxa < tmpmax:
                maxa = tmpmax
                maxi = i

        rect=cv2.minAreaRect(contours[maxi])
        box = cv2.boxPoints(rect)
        box = np.int0(box)

        x = box[1][0]
        y = box[2][1]
        w = box[0][0] - box[1][0]
        h = box[1][1] - box[2][1]
        result = img[y: y + h, x: x + w]
        return result

In [27]:
right = cv2.imread('demo/yard-right.png')
left = cv2.imread('demo/yard-left.png')
# resizing, so it's faster
#right = imutils.resize(right, width=600)
#left = imutils.resize(left, width=600)
res = Panorama.makePanorama(right, left, drawKeypoints=True, drawMatches=True, blend_type=Panorama.BOTH)

In [28]:
cv2.imwrite('demo/res1.png', Panorama.crop(res['resBlend']))
cv2.imwrite('demo/res2.png', Panorama.crop(res['resNoBlend']))
cv2.imwrite('demo/matches.png', res['matches'])
cv2.imwrite('demo/kpsL.png', res['kpsL'])

True